In [10]:
#imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
import plotly.express as px

%matplotlib inline
from sklearn import svm

In [11]:
df = pd.read_csv("top10s.csv")
df.head()
#df.info()

,Unnamed: 0,title,artist,top genre,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop
0,1,"Hey, Soul Sister",Train,neo mellow,2010,97,89,67,-4,8,80,217,19,4,83
1,2,Love The Way You Lie,Eminem,detroit hip hop,2010,87,93,75,-5,52,64,263,24,23,82
2,3,TiK ToK,Kesha,dance pop,2010,120,84,76,-3,29,71,200,10,14,80
3,4,Bad Romance,Lady Gaga,dance pop,2010,119,92,70,-4,8,71,295,0,4,79
4,5,Just the Way You Are,Bruno Mars,pop,2010,109,84,64,-5,9,43,221,2,4,78


In [12]:
#extracting columns we will use to base classifications
#dropping unnecessary columns

# UNCOMMENT FOR MEGADATASET
# df.drop(columns=['year', 'release_date', 'id', 'duration_ms', 'artists', 
#                  'name', 'explicit', 'popularity', 'mode', 'key', 'loudness'], inplace=True)

df.drop(columns=['Unnamed: 0','year','dur','pop'], inplace= True)
df

,title,artist,top genre,bpm,nrgy,dnce,dB,live,val,acous,spch
0,"Hey, Soul Sister",Train,neo mellow,97,89,67,-4,8,80,19,4
1,Love The Way You Lie,Eminem,detroit hip hop,87,93,75,-5,52,64,24,23
2,TiK ToK,Kesha,dance pop,120,84,76,-3,29,71,10,14
3,Bad Romance,Lady Gaga,dance pop,119,92,70,-4,8,71,0,4
4,Just the Way You Are,Bruno Mars,pop,109,84,64,-5,9,43,2,4
...,...,...,...,...,...,...,...,...,...,...,...
598,Find U Again (feat. Camila Cabello),Mark Ronson,dance pop,104,66,61,-7,20,16,1,3
599,Cross Me (feat. Chance the Rapper & PnB Rock),Ed Sheeran,pop,95,79,75,-6,7,61,21,12
600,"No Brainer (feat. Justin Bieber, Chance the Ra...",DJ Khaled,dance pop,136,76,53,-5,9,65,7,34
601,Nothing Breaks Like a Heart (feat. Miley Cyrus),Mark Ronson,dance pop,114,79,60,-6,42,24,1,7


In [13]:
fig = px.scatter(df, x='bpm', y='nrgy',color='top genre', hover_name='artist',hover_data=['title'])
fig.show()

In [23]:
k = 5

data = np.array_split(df, k) # split the data into 5 roughly equal groups of samples

results = [] # accuracy

# create the support vector machines with the linear, poly, and rbf kernels
clfs = [svm.SVC(kernel="linear"), svm.SVC(kernel="poly"), svm.SVC(kernel="rbf")]

for i in range(k):                                          # run 5 experiments for 5-fold cross-validation
    test_data_set = data[i]                                 # uses one group of data as the test data
    train_data_set = pd.concat(data[:i] + data[i+1:])       # train data takes the rest

    test_labels = np.array(test_data_set.iloc[:, 2])        # use the first column as the label
    test_data = np.array(test_data_set.iloc[:, 3:])         # use the rest for the data

    train_labels = np.array(train_data_set.iloc[:, 2])      # use the first column as the label
    train_data = np.array(train_data_set.iloc[:, 3:])       # use the rest for the data

    experiment_result = []      # the results using each svm for this partitioned data set

    for j in range(len(clfs)):  # use the data set to train each svm
        clfs[j].fit(train_data, train_labels)       # train the svm
        predictions = clfs[j].predict(test_data)    # predict
        accuracy = sum(predictions == test_labels) / len(test_labels)   # get accuracy
        experiment_result.append(accuracy)          # append to the experiment results

    results.append(experiment_result)   # append the row of data to the results

# put the results into a dataframe
results = pd.DataFrame(results)
results.columns = ["Linear", "Poly", "RBF"]

# calculate the average accuracy for each column
avg_accuracy = [sum(results.iloc[:, i]) / len(results) for i in range(3)]
results.loc["Avg"] = avg_accuracy

# display results
print(results)

       Linear      Poly       RBF
0    0.570248  0.603306  0.611570
1    0.537190  0.545455  0.545455
2    0.528926  0.537190  0.537190
3    0.508333  0.525000  0.516667
4    0.458333  0.500000  0.500000
Avg  0.520606  0.542190  0.542176
